In [1]:
import pandas as pd
import sys
sys.path.insert(0,r"C:\Users\LENOVO\Downloads")
import numpy as np
import csv
from sklearn.model_selection import train_test_split
import class_report as cr
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, auc, f1_score, log_loss
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
pd.set_option('display.max_row', None)
pd.set_option('display.max_column', None)

In [3]:
base_sur = pd.read_csv(r'C:\Users\LENOVO\Desktop\base_sur.txt', sep = ",", engine = "python")

In [4]:
base2_sur = base_sur[['SUSCNITC','SERVDESC','SESUCUSA','SESUCICL','SESULOCA',
                                  'TIPIFICACION_CLIENTE','CALIFICACION_VECTOR_SERVICIO',
                                  'CANAL_ENVIO','SEGMENTACION','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR',
                                  'ANTIGUEDAD_DIAS','CANAL_PAGO','SESUSAPE','SESUSAAN','PROM_SUSC']]
base3_sur = base2_sur.dropna()

In [5]:
tmp = base3_sur[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO']]
tmp.columns

group_nit = tmp.groupby(['SUSCNITC'])['CALIFICACION_VECTOR_SERVICIO'].mean().reset_index()
group_nit = group_nit.rename(columns={"CALIFICACION_VECTOR_SERVICIO":"PROM_CALIFICACION"})

In [6]:
base4_sur = pd.merge(base3_sur, group_nit, how='left', on=['SUSCNITC'])

In [7]:
base4_sur['y'] = pd.cut(x=base4_sur['PROM_CALIFICACION'],
                            bins=[-1,50,76,100],
                            labels=['No Pago','Pago Inoportuno','Pago'])

In [8]:
base5_sur = base4_sur[['SERVDESC','SESUCUSA','SESUCICL','SESULOCA','TIPIFICACION_CLIENTE',
                       'CALIFICACION_VECTOR_SERVICIO','CANAL_ENVIO','SEGMENTACION','ESTRATO_AGRUPADO',
                       'DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS','CANAL_PAGO','SESUSAPE','SESUSAAN','PROM_SUSC','y']]

In [9]:
names = base5_sur.columns

for x in names:
    globals()['FREQ_%s' % x]= base5_sur[x].value_counts().rename_axis([x]).reset_index(name='F_{x}'.format(x=x))

In [10]:
freq_sur = base5_sur.merge(FREQ_SERVDESC,on="SERVDESC").merge(FREQ_SESUCUSA,on='SESUCUSA').merge(FREQ_SESUCICL,on='SESUCICL').merge(FREQ_SESULOCA,on='SESULOCA').merge(FREQ_TIPIFICACION_CLIENTE,on='TIPIFICACION_CLIENTE').merge(FREQ_CANAL_ENVIO,on='CANAL_ENVIO').merge(FREQ_SEGMENTACION,on='SEGMENTACION').merge(FREQ_ESTRATO_AGRUPADO,on='ESTRATO_AGRUPADO').merge(FREQ_DEPARTAMENTO_AGR,on='DEPARTAMENTO_AGR').merge(FREQ_CANAL_PAGO,on='CANAL_PAGO')

In [11]:
freq_sur2 = freq_sur[['F_SERVDESC', 'F_SESUCUSA',
       'F_SESUCICL', 'F_SESULOCA', 'F_TIPIFICACION_CLIENTE', 'F_CANAL_ENVIO',
       'F_SEGMENTACION', 'F_ESTRATO_AGRUPADO', 'F_DEPARTAMENTO_AGR',
       'F_CANAL_PAGO']]

In [13]:
freq_sur2['AVG_FREQ'] = freq_sur2.mean(axis=1)

C:\Users\LENOVO\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
n, m = freq_sur2.shape
n_reg_del = round((0.05 * n) /100)
outliers = freq_sur2.nsmallest(n_reg_del, 'AVG_FREQ')
outliers_idx = outliers.index
outliers_idx = outliers_idx.sort_values(ascending=False)

In [14]:
for x in outliers_idx:
    base5_sur = base5_sur.drop([x])

In [15]:
#base5_sur = base5_sur.drop([332839])

In [16]:
base5_sur.shape

(332866, 16)

In [15]:
dataTypeSeries = base5_sur.dtypes
dataTypeSeries

cat = base5_sur.select_dtypes(include=['object','category'])
cuanti = base5_sur.select_dtypes(include=['int64','float64'])
  
cat = cat.apply(lambda col: LabelEncoder().fit_transform(col))
base5_sur = cuanti.join(cat)

In [16]:
X = base5_sur[['SERVDESC','SESUCUSA','SESUCICL','SESULOCA','TIPIFICACION_CLIENTE','CANAL_ENVIO',
               'SEGMENTACION','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS','CANAL_PAGO',
               'SESUSAPE','SESUSAAN','PROM_SUSC']]

y = base5_sur['y']

In [17]:
from datetime import datetime
instanteInicial = datetime.now()
X_ = StandardScaler().fit_transform(X)
#X_ = X
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2,random_state=321)

In [19]:
classifiers = [
    KNeighborsClassifier(3),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=500),
    LogisticRegression(multi_class="multinomial", solver="lbfgs", C=5),
    GaussianNB()
    ]

# Competencia de clasificadores
log_cols=["Classifier", "Accuracy", "Log Loss", "F1_SCORE"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__  
    print("="*30)
    print(name)
    print('****Resultados****')    
    train_predictions = clf.predict_proba(X_test)
    acc = cr.class_report(y_true = y_test, 
                                y_pred = clf.predict(X_test),
                                y_score = train_predictions)
    print("Métricas: {}".format(acc))
    log_entry = pd.DataFrame([[name, acc]])
    log = log.append(log_entry)
    
print("="*30)

KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.938936  0.942902  0.940915  21069.0  21158.0  0.978620
1             0.954703  0.970418  0.962496  29038.0  29516.0  0.985369
2             0.899623  0.868646  0.883863  16467.0  15900.0  0.958085
avg / total   0.936089  0.936537  0.936216  66574.0  66574.0  0.977083
DecisionTreeClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.937145  0.963121  0.949956  21069.0  21653.0  0.984923
1             0.970102  0.981094  0.975567  29038.0  29367.0  0.990798
2             0.945866  0.893423  0.918897  16467.0  15554.0  0.970658
avg / total   0.953677  0.953721  0.953444  66574.0  66574.0  0.985971
RandomForestClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.897334  0.801889  0.846931  21069.0  18828.0  0.9

In [22]:
kf = StratifiedKFold(n_splits=5,shuffle=False)

accuracy_model = []
# Iteraciones del entrenamiento sobre los kfolds
#X_fin = X[['SESUSERV','SESUCUSA', 'SESUSAPE','SESUCICL', 'TIPIFICACION_CLIENTE', 'PROM_SUSC',
#'CANAL_ENVIO','SEGMENTACION', 'ESTRATO_AGRUPADO','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]
X_fin = X
X_ = StandardScaler().fit_transform(X_fin)

for train_index, test_index in kf.split(X_, y) :
    # Particion train test para validacion cruzada
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Entrenamiento de la red MLP
    model = KNeighborsClassifier(7).fit(X_train, y_train) 
    accuracy_model.append(cr.class_report(y_true = y_test, 
                                       y_pred = model.predict(X_test),
                                      y_score = model.predict_proba(X_test)))
 
# Print the accuracy    
print(accuracy_model)

[             precision    recall  f1-score  support     pred       AUC
0             0.880294  0.874001  0.877136  21262.0  21110.0  0.953348
1             0.814976  0.891775  0.851648  29023.0  31758.0  0.929081
2             0.670937  0.564759  0.613286  16322.0  13739.0  0.829969
avg / total   0.800530  0.805966  0.801374  66607.0  66607.0  0.920429,              precision    recall  f1-score  support     pred       AUC
0             0.863133  0.890739  0.876719  21261.0  21941.0  0.953219
1             0.821412  0.875271  0.847487  29023.0  30926.0  0.924333
2             0.682824  0.574772  0.624156  16323.0  13740.0  0.837610
avg / total   0.800767  0.806567  0.802087  66607.0  66607.0  0.919360,              precision    recall  f1-score  support     pred       AUC
0             0.878080  0.883407  0.880735  21262.0  21391.0  0.956285
1             0.792617  0.882606  0.835195  29022.0  32317.0  0.903601
2             0.665400  0.525822  0.587434  16323.0  12899.0  0.792961
avg

In [23]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, model.predict(X_test))

array([[18899,   839,  1524],
       [  744, 25606,  2672],
       [ 2278,  4652,  9392]], dtype=int64)